In [ ]:
%%writefile requirements.txt
boto3
strands-agents
strands-agents[sagemaker]
strands-agents-tools
bedrock-agentcore
bedrock-agentcore-starter-toolkit
openai

In [ ]:
%pip install -r requirements.txt --upgrade

# Creating an AgentCore Runtime app from SageMaker Studio

Amazon Bedrock AgentCore Runtime provides a secure, serverless and purpose-built hosting environment for deploying and running AI agents or tools. It offers a lot of benefits, which you can learn about in the [Host agent or tools with Amazon Bedrock AgentCore Runtime](https://docs.aws.amazon.com/bedrock-agentcore/latest/devguide/agents-tools-runtime.html) documentation.

![building-agents-in-sagemaker-studio.png](images/building-agents-in-sagemaker-studio.png)

In this notebook, you will learn how to use SageMaker Studio to:

1. Create an AI Agent with [Strands Agents](https://strandsagents.com/) powered by a SageMaker AI endpoint
2. Configure the agent to run on Amazon Bedrock AgentCore Runtime
    - Using the Bedrock Agentcore Starter Toolkit SDK
    - Directly creating an ECR image with Amazon SageMaker Studio Image Build CLI - not shown in this notebook, refer to the [Using the Amazon SageMaker Studio Image Build CLI to build container images from your Studio JupyterLab notebooks](https://aws.amazon.com/blogs/machine-learning/using-the-amazon-sagemaker-studio-image-build-cli-to-build-container-images-from-your-studio-notebooks/) blog
4. Launch it and invoke it

## Requirements

1. A model deployed on a **SageMaker AI Endpoint** - in our case, Qwen 3 32B. Instance type is irrelevant. If you don't know where to start, we suggest deploying a model from **SageMaker JumpStart** - learn more [here](https://aws.amazon.com/blogs/machine-learning/qwen3-family-of-reasoning-models-now-available-in-amazon-bedrock-marketplace-and-amazon-sagemaker-jumpstart/)
1. A **SageMaker AI Execution Role** with broad enough permissions, including create/delete IAM Role, create/delete ECR repository, start CodeBuild projects
1. **Docker enabled on SageMaker Studio** - this needs to be enabled both at the level of the Domain and the User profile (required for Local build of the image)

![image.png](images/sm-studio-docker-permissions.png)

In [ ]:
import os
SAGEMAKER_ENDPOINT_NAME = os.environ.get("SM_AI_ENDPOINT", "jumpstart-dft-hf-reasoning-qwen3-32-20250804-154835")

In [ ]:
%%writefile agent.py
from bedrock_agentcore.runtime import BedrockAgentCoreApp
from strands import Agent
from strands.models.sagemaker import SageMakerAIModel
import os

SAGEMAKER_ENDPOINT_NAME = os.environ.get("SM_AI_ENDPOINT", "jumpstart-dft-hf-reasoning-qwen3-32-20250804-154835")

app = BedrockAgentCoreApp()
agent = Agent()
model = SageMakerAIModel(
    endpoint_config={"endpoint_name": SAGEMAKER_ENDPOINT_NAME},
    payload_config={"max_tokens": 1024*5, "stream": True}
)

@app.entrypoint
def invoke(payload):
    """Process user input and return a response"""
    user_message = payload.get("prompt", "Hello")
    result = agent(user_message)
    return {"result": result.message}

if __name__ == "__main__":
    app.run()

## Local Testing

In [ ]:
import nest_asyncio
nest_asyncio.apply()

In [ ]:
!python agent.py

Now, go to the Console in SageMaker AI and run:

```bash
curl -X POST http://localhost:8080/invocations \
-H "Content-Type: application/json" \
-d '{"prompt": "Hello world!"}'
```

## Deployment

In [ ]:
from bedrock_agentcore_starter_toolkit import Runtime
import boto3
from sagemaker.utils import name_from_base


agent_name = "hello_world_sagemaker"

agentcore_runtime = Runtime()
response = agentcore_runtime.configure(
    agent_name=agent_name,
    region=boto3.Session().region_name,
    auto_create_ecr=True,   # Instructs Runtime to auto-create ECR repo
    auto_create_execution_role=True,  # Instructs Runtime to auto-create execution role
    entrypoint="agent.py", # The core of the agent
    requirements_file="requirements.txt", # dependencies
)

launch_result = agentcore_runtime.launch(auto_update_on_conflict=True)
print("Launch completed ✓")
print(f"Agent ARN: {launch_result.agent_arn}")
print(f"Agent ID: {launch_result.agent_id}")

## Add the SageMaker AI permissions

Since we need to have the agent invoke the SageMaker AI Endpoint, we'll need to add the right permissions.

In [ ]:
agentcore_client = boto3.client('bedrock-agentcore-control')
execution_role_arn = agentcore_client.get_agent_runtime(agentRuntimeId=launch_result.agent_id)["roleArn"]
execution_role_name = execution_role_arn.split('/')[-1]

In [ ]:
import json

region_name = boto3.Session().region_name
account_id = boto3.client("sts").get_caller_identity()["Account"]
# Get the Agent Execution Role ARN and attach the SageMake Invoke Endpoint policy
iam_client = boto3.client("iam")
permissions_policy = {
  "Version": "2012-10-17",
  "Statement": [
    {
      "Effect": "Allow",
      "Action": [
        "sagemaker:InvokeEndpoint", "sagemaker:InvokeEndpointAsync",
        "sagemaker:InvokeEndpointWithResponseStream"
      ],
      "Resource": f"arn:aws:sagemaker:{region_name}:{account_id}:endpoint/{SAGEMAKER_ENDPOINT_NAME}"
    }
  ]
}
iam_client.put_role_policy(
    RoleName=execution_role_name,
    PolicyName="sagemaker-invoke-endpoint",
    PolicyDocument=json.dumps(permissions_policy)
)

In [ ]:
import time
status_response = agentcore_runtime.status()
status = status_response.endpoint['status']
end_status = ['READY', 'CREATE_FAILED', 'DELETE_FAILED', 'UPDATE_FAILED']
while status not in end_status:
    time.sleep(10)
    status_response = agentcore_runtime.status()
    status = status_response.endpoint['status']
    print(status)
status

In [ ]:
invoke_response = agentcore_runtime.invoke({"prompt": "Hello world from SageMaker AI!"})
json.loads(invoke_response['response'][0])

## Clean up

In [ ]:
launch_result.ecr_uri, launch_result.agent_id, launch_result.ecr_uri.split('/')[1]

In [ ]:
import boto3


boto_session = boto3.Session()

agentcore_control_client = boto_session.client('bedrock-agentcore-control')
agentcore_control_client.delete_agent_runtime(agentRuntimeId=launch_result.agent_id)

ecr_client = boto_session.client('ecr')
ecr_client.delete_repository(
    repositoryName=launch_result.ecr_uri.split('/')[1],
    force=True
)

iam_client = boto_session.client("iam")
policies = iam_client.list_role_policies(RoleName=execution_role_name)['PolicyNames']
for policy in policies:
    iam_client.delete_role_policy(RoleName=execution_role_name, PolicyName=policy)
iam_client.delete_role(RoleName=execution_role_name)